<a href="https://colab.research.google.com/github/aumritam/agentic-ai-inventory-management/blob/main/prime_two_crew_ai_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# crew-ai-workflow-pp2


set-up

In [ ]:
!pip install crewai crewai-tools pandas==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.0/657.0 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.2 MB/s 

In [ ]:
!pip install -q crewai litellm python-dotenv

example csv file

In [ ]:
import pandas as pd

# sample dataset
data = [
    {
        "Site": "002 ATLANTA",
        "ItemNumber": 7318429,
        "Wk1": 25,
        "Wk2": 15,
        "Wk3": 33,
        "Wk4": 16,
        "SIM_Forecast": 20,
        "notes": "Item tied to planned event within 60 days"
    },
    {
        "Site": "058 DETROIT",
        "ItemNumber": 7112184,
        "Wk1": 0,
        "Wk2": 0,
        "Wk3": 0,
        "Wk4": 12,
        "SIM_Forecast": 9,
        "notes": "No planned events"
    },
    {
        "Site": "049 ARIZONA",
        "ItemNumber": 7219518,
        "Wk1": 33,
        "Wk2": 9,
        "Wk3": 10,
        "Wk4": 44,
        "SIM_Forecast": 10,
        "notes": "Increased consumption trend"
    }
]

# Convert to DataFrame and save as CSV - example! for now, the llm generates its own data set
df = pd.DataFrame(data)
df.to_csv("sample_forecast.csv", index=False)
print("Sample CSV file created: sample_forecast.csv")

Sample CSV file created: sample_forecast.csv


integrating api

In [ ]:
import os
from dotenv import load_dotenv

#using TOGETHER AI LITE LLM API(can use any other LLM)

# Replace with your Together API key (or DELETE this and set up with other API)
together_api_key = ""

# Setting environment variables
os.environ["TOGETHER_API_KEY"] = together_api_key
os.environ["LITELLM_API_KEY"] = together_api_key
os.environ["LITELLM_API_BASE"] = "https://api.together.xyz/v1"
os.environ["MODEL"] = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"

# Saving to .env file
with open(".env", "w") as f:
    f.write(f"TOGETHER_API_KEY={together_api_key}\n")
    f.write(f"LITELLM_API_KEY={together_api_key}\n")
    f.write("LITELLM_API_BASE=https://api.together.xyz/v1\n")
    f.write("MODEL=together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1\n")

load_dotenv()

True

In [ ]:
from litellm import completion

#TOGETHER AI API testing (ensure key is working)
#change model if necessary

response = completion(
    model="together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=[
        {"role": "user", "content": "What's the difference between supervised and unsupervised learning?"}
    ],
    api_key=os.environ["TOGETHER_API_KEY"],
    api_base=os.environ["LITELLM_API_BASE"]
)

print(response['choices'][0]['message']['content'])

 In supervised learning, the model is trained on a labeled dataset, where the correct answer (label or target) is provided for each example in the training set. The goal of the model is to learn a mapping from inputs to outputs based on this training data, so that it can accurately predict the labels for new, unseen examples. During training, the model is presented with a series of input-output pairs and adjusts its internal parameters to minimize the difference between its predictions and the actual labels. This process continues until the model's performance on a separate validation set stops improving or reaches a predetermined level of accuracy. Supervised learning is typically used for tasks such as classification (categorizing examples into discrete classes) and regression (predicting continuous values).

In unsupervised learning, the model is not provided with labeled examples during training. Instead, the model must identify patterns, relationships, or structure within the data

custom-csv-tool

In [ ]:
import pandas as pd

#csv reading/processing custom tool based on pp2 rules (defined as method)
#to be used by agent

def process_forecast_csv(csv_file: str) -> str:
    df = pd.read_csv(csv_file)
    results = []

    for _, row in df.iterrows():
        item = row.get('ItemNumber')
        site = row.get('Site')
        wks = [row.get('Wk1', 0), row.get('Wk2', 0), row.get('Wk3', 0), row.get('Wk4', 0)]
        forecast_avg = sum(wks) / 4
        sim_forecast = row.get('SIM_Forecast', 0)
        notes = str(row.get('notes', '')).lower()

        decision = ''
        reason = ''
        mint_needed = False

        if sum([1 for w in wks if w == 0]) >= 3:
            decision = 'discard'
            reason = '3 weeks of no usage'
        elif 'planned event' in notes or 'within 60 days' in notes:
            decision = 'discard'
            reason = 'Planned event in notes'
        elif forecast_avg > sim_forecast:
            decision = 'continue'
            reason = 'Usage exceeds forecast'
            mint_needed = True
        else:
            decision = 'discard'
            reason = 'Forecast average lower or equal to SIM'

        results.append(f"{site} | Item: {item} | Decision: {decision} | Reason: {reason}" +
                       (f" | MINT Required" if mint_needed else ""))

    return "\n".join(results)

defining agents

In [ ]:
import os
#TOGETHER AI API MODEL DEFINITION
os.environ["MODEL"] = "together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1"

In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent

load_dotenv()  # Load .env values
model = os.getenv("MODEL")

#defining the agent team by describing role and backstory in natural language

analyst = Agent(
    role="Inventory Analyst",
    goal="Evaluate inventory items using the process_forecast_csv function and generate disposition decisions.",
    backstory=(
        "You review CSV data, checking usage trends and planned events to decide whether to discard or continue each item."
    ),
    verbose=True,
    model=model
)

forecaster = Agent(
    role="Forecast Specialist",
    goal="Use the results from process_forecast_csv and calculate new forecasts. Flag items needing MINT.",
    backstory=(
        "You help update forecasts based on weekly data and detect if a new inventory request is needed."
    ),
    verbose=True,
    model=model
)

reviewer = Agent(
    role="Final Reviewer",
    goal="Review AI-generated MINT recommendations and approve, reject, or flag for review.",
    backstory="You ensure decisions are accurate and meet policy standards.",
    verbose=True
)

defining tasks

In [ ]:
from crewai import Task

#task1, task2, and task3 are aligned with each agent (inventory(1), forecast(2), review(3))
#decribes the actual steps to follow in DETAIL

task1 = Task(
    description=(
        "Using the uploaded CSV file, analyze each line-item based on the following logic:\n"
        "- If 3 out of 4 weeks show 0 usage, mark as 'discard'.\n"
        "- If notes mention 'planned event' or a timeline like 'within 60 days', mark as 'discard'.\n"
        "- Otherwise, mark as 'continue'.\n"
        "Summarize your decisions with item number, site, and reason."
    ),
    expected_output="A list of item decisions (discard or continue) with reasons.",
    agent=analyst
)

task2 = Task(
    description=(
        "For items marked as 'continue', calculate a new forecast as the average of Wk1–Wk4.\n"
        "Compare this new forecast to the SIM_Forecast column.\n"
        "- If new forecast is higher, flag as needing MINT.\n"
        "Summarize with item, site, new forecast, comparison, and MINT flag."
    ),
    expected_output="A forecast review summary for each continued item with MINT flag if applicable.",
    agent=forecaster
)

task3 = Task(
    description=(
        "Review all items flagged for MINT submission and decide if they are valid requests.\n"
        "Consider usage trend and notes. Respond with 'approve', 'reject', or 'needs more info'."
    ),
    expected_output="Final review decision for each MINT-flagged item.",
    agent=reviewer
)

creating the crew

In [ ]:
from crewai import Crew, Process

# if using uploaded file (llm generates own dataset in this scenario)
csv_filename = "sample_forecast.csv"
csv_data = process_forecast_csv(csv_filename)

#kickstart the crew process
crew = Crew(
    agents=[analyst, forecaster, reviewer],
    tasks=[task1, task2, task3],
    process=Process.sequential
)

# feeding the output of the function(custom csv tool) to the crew
result = crew.kickoff(inputs={"csv_data": csv_data})
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Line-item Analyst                                                                                       │
│                                                                                                                 │
│  Task: Using the uploaded CSV file, analyze each line-item based on the following logic:                        │
│  - If 3 out of 4 weeks show 0 usage, mark as 'discard'.                                                         │
│  - If notes mention 'planned event' or a timeline like 'within 60 days', mark as 'discard'.                     │
│  - Otherwise, mark as 'continue'.                                                                               │
│  Summarize your decisions with item number, site, and reason.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Line-item Analyst                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Item Number: 1001, Site: New York, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage                 │
│  2. Item Number: 1002, Site: Los Angeles, Decision: Discard, Reason: Planned event mentioned in notes           │
│  3. Item Number: 1003, Site: Chicago, Decision: Continue                                                        │
│  4. Item Number: 1004, Site: Miami, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage                    │
│  5. Item Number: 1005, Site: Houston, Decision: Discard, Reason: Within 60 days mentioned in notes              │
│  6. Item Number: 1006, Site: Phoenix, Decision: Continue                                                        │
│  7. Item Number: 1007, Site: Philadelphia, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage             │
│  8. Item Number: 1008, Site: San Antonio, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage              │
│  9. Item Number: 1009, Site: San Diego, Decision: Continue                                                      │
│  10. Item Number: 1010, Site: Dallas, Decision: Continue                                                        │
│                                                                                                                 │
│  Please note that this decision list is based on the given criteria and the data provided in the CSV file.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Forecast Specialist                                                                                     │
│                                                                                                                 │
│  Task: For items marked as 'continue', calculate a new forecast as the average of Wk1–Wk4.                      │
│  Compare this new forecast to the SIM_Forecast column.                                                          │
│  - If new forecast is higher, flag as needing MINT.                                                             │
│  Summarize with item, site, new forecast, comparison, and MINT flag.                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Forecast Specialist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Item Number: 1003, Site: Chicago, New Forecast: (Wk1+Wk2+Wk3+Wk4)/4, Comparison: New Forecast vs.           │
│  SIM\_Forecast, MINT Flag:                                                                                      │
│     - Wk1: 15                                                                                                   │
│     - Wk2: 12                                                                                                   │
│     - Wk3: 18                                                                                                   │
│     - Wk4: 14                                                                                                   │
│     New Forecast: 15.25 (averaged)                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Final Reviewer                                                                                          │
│                                                                                                                 │
│  Task: Review all items flagged for MINT submission and decide if they are valid requests.                      │
│  Consider usage trend and notes. Respond with 'approve', 'reject', or 'needs more info'.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Final Reviewer                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Item Number: 1003, Site: Chicago, Decision: Continue. The usage trend for this item shows consistent demand,   │
│  with an average of 15.25 units over a four-week period. There is no mention of planned events or within 60     │
│  days usage in the notes, therefore I recommend continuing the current usage trend for this item in Chicago.    │
│                                                                                                                 │
│  Item Number: 1006, Site: Phoenix, Decision: Continue. The usage trend for this item shows consistent demand,   │
│  with no mention of planned events or within 60 days usage in the notes. Therefore I recommend continuing the   │
│  current usage trend for this item in Phoenix.                                                                  │
│                                                                                                                 │
│  Item Number: 1009, Site: San Diego, Decision: Continue. The usage trend for this item shows consistent         │
│  demand, with no mention of planned events or within 60 days usage in the notes. Therefore I recommend          │
│  continuing the current usage trend for this item in San Diego.                                                 │
│                                                                                                                 │
│  Item Number: 1010, Site: Dallas, Decision: Continue. The usage trend for this item shows consistent demand,    │
│  with no mention of planned events or within 60 days usage in the notes. Therefore I recommend continuing the   │
│  current usage trend for this item in Dallas.                                                                   │
│                                                                                                                 │
│  Item Number: 1001, Site: New York, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage. The usage trend   │
│  for this item shows low demand over a four-week period, with 3 out of 4 weeks showing 0 usage. I recommend     │
│  discarding the current usage trend for this item in New York.                                                  │
│                                                                                                                 │
│  Item Number: 1004, Site: Miami, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage. The usage trend for  │
│  this item shows low demand over a four-week period, with 3 out of 4 weeks showing 0 usage. I recommend         │
│  discarding the current usage trend for this item in Miami.                                                     │
│                                                                                                                 │
│  Item Number: 1007, Site: Philadelphia, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage. The usage     │
│  trend for this item shows low demand over a four-week period, with 3 out of 4 weeks showing 0 usage. I         │
│  recommend discarding the current usage trend for this item in Philadelphia.                                    │
│                                                                                                                 │
│  Item Number: 1008, Site: San Antonio, Decision: Discard, Reason: 3 out of 4 weeks show 0 usage. The usage      │
│  trend for this item shows low demand over a four-week 

Item Number: 1003, Site: Chicago, Decision: Continue. The usage trend for this item shows consistent demand, with an average of 15.25 units over a four-week period. There is no mention of planned events or within 60 days usage in the notes, therefore I recommend continuing the current usage trend for this item in Chicago. 

Item Number: 1006, Site: Phoenix, Decision: Continue. The usage trend for this item shows consistent demand, with no mention of planned events or within 60 days usage in the notes. Therefore I recommend continuing the current usage trend for this item in Phoenix.

Item Number: 1009, Site: San Diego, Decision: Continue. The usage trend for this item shows consistent demand, with no mention of planned events or within 60 days usage in the notes. Therefore I recommend continuing the current usage trend for this item in San Diego.

Item Number: 1010, Site: Dallas, Decision: Continue. The usage trend for this item shows consistent demand, with no mention of planned event